In [ ]:
pip install -U urllib3 requests

In [ ]:
pip install selenium

In [3]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time

In [4]:
def getReviews(page_url):
    response = requests.get(page_url)
    source = response.text
    soup = BeautifulSoup(source, "html.parser")
    
    names = []
    dates = []
    contents = []
    authors = []
    scores = []
    
    
    name=soup.find_all('h1', attrs={'data-component' : 'ProductName'})
    if len(name)!=0:
        names.append(name[0].text)
    else:
        names.append("Null")
    
    date=soup.find_all('span', attrs={'class' : 'Bdstartw(1px) Bdstarts(s) Bdstartc(#6b6d70) Pstart(6px) C(engadgetFontDarkGray)'})
    if len(date)!=0:
        dates.append(date[0].text)
    else:
        dates.append("Null")
    
    content=soup.find_all('p', attrs={'class' : 'M(0) Fz(20px) Fz(16px)!--md Lh(32px) Lh(28px)--lg Lh(24px)--md serif'})
    if len(content)!=0:
        contents.append(content[0].text)
    else:
        contents.append("Null") 
    
    author=soup.find_all('a', attrs={'class' : 'Pend(6px) Ai(c) Td(n) Cur(p) C(engadgetFontDarkPurple)'})
    if len(author)!=0:
        authors.append(author[0].text)
    else:
        authors.append("Null")
    
    score=soup.find_all('div', attrs={'class' : 'Pos(a) B(0) Start(0) Pstart(8px) Pb(10px) Fz(24px) Fw(700) C(white)'})
    if len(score)!=0:
        scores.append(score[0].text)
    else:
        scores.append("Null")
    
    
    
    reviews = pd.DataFrame({
        'Product Name': names,
        'Date': dates,
        'Review': contents,
        'Author': authors,
        'Rating': scores,
        })
    # Returning revies data frame
    return reviews

In [9]:
def getAllReviews(page_url):
    
    driver = webdriver.Chrome('C:/Users/janha/Downloads/chromedriver.exe')
    driver.get(page_url)
    
    for i in range(0,12):
        path=driver.find_element_by_id('LoadMore')
        path.click()
        time.sleep(2)
        

    temp=driver.current_url
    response = requests.get(temp)
    source = response.text
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    urls=[]
    names = []
    dates = []
    contents = []
    authors = []
    scores = []
    reviews = pd.DataFrame({
        'Product Name': names,
        'Date': dates,
        'Review': contents,
        'Author': authors,
        'Rating': scores,
        })
    

    
    url=soup.find_all('a', attrs={'class' : 'StretchedBox'})
    for i in url:
        temp=i.get('href')
        temp='https://www.engadget.com'+temp
        urls.append(temp)
    for j in urls:
        r=getReviews(j)
        reviews = reviews.append(r, ignore_index = True)
    return reviews
        

In [10]:
page_url = 'https://www.engadget.com/products/phones/'
reviews=getAllReviews(page_url)
reviews.head(200)

<ipython-input-9-7c86a17d4947>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/janha/Downloads/chromedriver.exe')
<ipython-input-9-7c86a17d4947>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  path=driver.find_element_by_id('LoadMore')


,Product Name,Date,Review,Author,Rating
0,Sony Xperia 1 III,"October 27, 2021",The Xperia 1 III is another Sony phone aimed a...,Mat Smith,81
1,Google Pixel 6 Pro,"October 25, 2021","For hundreds less than the competition, the Pi...",Cherlynn Low,91
2,Google Pixel 6,"October 25, 2021",The Pixel 6 is hands-down the best Android pho...,Cherlynn Low,89
3,Microsoft Surface Duo 2,"October 21, 2021",Upgraded hardware and a new Glance Bar don’t m...,Cherlynn Low,63
4,Motorola Edge (2021),"October 12, 2021",The 2021 Motorola Edge isn’t without compromis...,Igor Bonifacic,82
...,...,...,...,...,...
139,Apple iPhone 11,"September 17, 2019",The iPhone 11 takes almost everything that mak...,Chris Velazco,91
140,Vivo NEX 3 5G,Null,We've added this product to our database but w...,Null,Null
141,Huawei Mate 30,Null,We've added this product to our database but w...,Null,Null
142,Huawei Mate 30 Pro,Null,We've added this product to our database but w...,Null,Null


In [ ]:
reviews.to_csv('CSVs/engadget.csv')